In [ ]:
import collections
import itertools
import subprocess

import matplotlib.pyplot as plt
import numpy as np

%matplotlib inline

# Centers figures
from IPython.core.display import HTML
HTML("""
<style>
.output_png {
    display: table-cell;
    text-align: center;
    vertical-align: middle;
}
</style>
""")

In [ ]:
args = ["MCTSv3AI 5", "ArthurAI"]

In [ ]:
output = "results/" + " ".join(args) + ".txt"
args = ["".join(arg.split()) for arg in args]
players = ["Player {}: {}".format(i + 1, x) for i, x in enumerate(args)]

In [ ]:
points = collections.defaultdict(list)
low_size = []
mid_size = []
high_size = []
rounds = []
wins = collections.defaultdict(list)


def parse(output_file):
    global wins
    with open(output_file) as file:
        line = file.readline().strip()
        while line:

            # Players
            assert line == "Players"
            pts = collections.defaultdict(int)
            line = file.readline().strip()
            while line != "Decks":
                parts = line.rpartition(" ")
                pts[parts[0]] = int(parts[2])
                line = file.readline().strip()
            for name in players:
                points[name].append(pts[name])

            # Decks
            assert line == "Decks"
            low_size.append(int(file.readline().strip().split()[1]))
            mid_size.append(int(file.readline().strip().split()[1]))
            high_size.append(int(file.readline().strip().split()[1]))

            # Rounds
            assert file.readline().strip() == "Rounds"
            rounds.append(int(file.readline().strip()))

            # Winner(s)
            assert file.readline().strip() == "Winner(s)"
            winners = set()
            line = file.readline().strip()
            while line and line != "Players":
                winners.add(line)
                line = file.readline().strip()
            for name in players:
                wins[name].append(1 if name in winners else 0)

    wins = {name: list(itertools.accumulate(wins[name])) for name in wins}
    return len(rounds)


n = parse(output)

In [ ]:
def moving_average(period):
    # https://stackoverflow.com/questions/13728392/moving-average-or-running-mean/43200476#43200476
    from scipy.ndimage.filters import uniform_filter1d
    return lambda x: uniform_filter1d(np.array(x, dtype=float), size=period)


smooth = moving_average(5)

In [ ]:
plt.figure(figsize=(16, 4))
for name in players:
    plt.plot(range(n), points[name], label=name)
plt.xlabel("Iteration number")
plt.ylabel("Number of points")
plt.grid()
plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(16, 4))
for name in players:
    plt.plot(range(n), smooth(points[name]), label=name)
plt.xlabel("Iteration number")
plt.ylabel("Number of points, smoothed")
plt.grid()
plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(16, 4))
plt.plot(range(n), low_size, label="Cards left in LOW deck")
plt.plot(range(n), mid_size, label="Cards left in MID deck")
plt.plot(range(n), high_size, label="Cards left in HIGH deck")
plt.plot(range(n), rounds, label="Number of rounds")
plt.xlabel("Iteration number")
plt.ylabel("Data")
plt.grid()
plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(16, 4))
plt.plot(range(n), smooth(low_size), label="Cards left in LOW deck")
plt.plot(range(n), smooth(mid_size), label="Cards left in MID deck")
plt.plot(range(n), smooth(high_size), label="Cards left in HIGH deck")
plt.plot(range(n), smooth(rounds), label="Number of rounds")
plt.xlabel("Iteration number")
plt.ylabel("Data, smoothed")
plt.grid()
plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(16, 4))
for name in players:
    plt.plot(range(n), wins[name], label=name)
plt.xlabel("Iteration number")
plt.ylabel("Cumulative number of wins")
plt.grid()
plt.legend()
plt.show()

In [ ]:
for name in players:
    print("{} wins: {}".format(name, wins[name][-1]))